### Time consuming of semblance

The main problem of semblance calculation is thar it very time consuming opeartion. This notebook is aimed to show few approaches to semblance calculation and redusing time.

In [1]:
import sys

import numpy as np
import matplotlib.pyplot as plt

sys.path.append('..')

from seismicpro.batchflow import ImagesBatch, Dataset, V, B, Pipeline, D
from seismicpro import FieldIndex, CustomIndex, SeismicDataset, seismic_plot, SeismicBatch

In [2]:
path_raw = '/data/H2_PAL/SEMB/H2_PAL_va_supergather_11.sgy'

index_raw = FieldIndex(name='raw', extra_headers=['offset', 'CDP'], path=path_raw)
ix_raw = CustomIndex(index_raw, index_name='CDP')

### Semblance calculation

For semblance calculation we use the following formula:
$$S = \frac{\sum^{k+N/2}_{k-N/2}(\sum^M_1 f_{ij})^2}{M \sum^{k+N/2}_{k-N/2}\sum^M_1 (f_{ij})^2} \text{ ,where }$$

* k - time sample
* N - window size
* M - number of traces
* f - value of amplitude

This secotion will contains few approaches for semblance calculation in order to reduce the time.

Frist approach is numba with 3 nested loops. Function ```_calc_semb_hard``` in utils that one can find [here](../seismicpro/src/semblance_utils.py).

In [3]:
dset = SeismicDataset(ix_raw, batch_class=SeismicBatch)

In [4]:
pipeline = (
    dset.p
    .load(fmt='segy', components='raw', tslice=slice(1500))
    .sort_traces(src='raw', dst='raw', sort_by='offset')
    .calculate_semblance('raw', 'semblance_hard', [1200, 6000], 
                         30, window=51, method='hard')
)
pipeline.run(1, n_iters=1, shuffle=1980, profile=True);

In [5]:
pipeline.elapsed_time

4.427786350250244

In [6]:
pipeline.show_profile_info(per_iter=True, detailed=False)

total_time  pipeline_time         batch_id
iter action                                                            
1    calculate_semblance #2    4.211385       4.198370  139795398802848
     load #0                   0.169323       0.094420  139795398802848
     sort_traces #1            0.023449       0.016205  139795398802848

In [8]:
pipeline.show_profile_info(per_iter=True, detailed=True)

ncalls  \
iter action                 id                                                           
1    calculate_semblance #2 <method 'acquire' of '_thread.lock' objects>::/...       4   
                            <method 'acquire' of '_thread.lock' objects>::/...       1   
                            wait::../seismicpro/batchflow/batchflow/decorat...       1   
                            wrapped_method::../seismicpro/batchflow/batchfl...       1   
                            _call_post_fn::../seismicpro/batchflow/batchflo...       1   
                            remove::../seismicpro/batchflow/batchflow/decor...       1   
                            wrap_with_threads::../seismicpro/batchflow/batc...       1   
                            shutdown::/usr/lib/python3.6/concurrent/futures...       1   
                            calculate_semblance::../seismicpro/batchflow/ba...       1   
                            _action_wrapper::../seismicpro/batchflow/batchf...       1   
     load #0                <method 'acquire' of '_thread.lock' objects>::/...       4   
                            <built-in method numpy.core._multiarray_umath.i...      27   
                            _load_segy::../seismicpro/src/seismic_batch.py:...       1   
                            <method 'factorize' of 'pandas._libs.hashtable....      12   
                            <method 'get_loc' of 'pandas._libs.index.BaseMu...      93   
                            <built-in method builtins.getattr>::/usr/local/...    2187   
                            _check::~::0::<built-in method builtins.isinsta...    2187   
                            <built-in method builtins.isinstance>::/usr/loc...     530   
                            _engine::/usr/local/lib/python3.6/dist-packages...      10   
                            find::/usr/local/lib/python3.6/dist-packages/pa...     329   
     sort_traces #1         <method 'acquire' of '_thread.lock' objects>::/...       4   
                            <method 'acquire' of '_thread.lock' objects>::/...       1   
                            <built-in method _thread.start_new_thread>::/us...       1   
                            wrap_with_threads::../seismicpro/batchflow/batc...       1   
                            wait::../seismicpro/batchflow/batchflow/decorat...       1   
                            wait::/usr/lib/python3.6/threading.py::533::wait         2   
                            _adjust_thread_count::/usr/lib/python3.6/concur...       1   
                            notify::/usr/lib/python3.6/queue.py::115::put            2   
                            wrapped_method::../seismicpro/src/seismic_batch...       1   
                            start::/usr/lib/python3.6/concurrent/futures/th...       1   

                                                                                 tottime  \
iter action                 id                                                             
1    calculate_semblance #2 <method 'acquire' of '_thread.lock' objects>::/...  4.196443   
                            <method 'acquire' of '_thread.lock' objects>::/...  0.000164   
                            wait::../seismicpro/batchflow/batchflow/decorat...  0.000119   
                            wrapped_method::../seismicpro/batchflow/batchfl...  0.000101   
                            _call_post_fn::../seismicpro/batchflow/batchflo...  0.000092   
                            remove::../seismicpro/batchflow/batchflow/decor...  0.000083   
                            wrap_with_threads::../seismicpro/batchflow/batc...  0.000079   
                            shutdown::/usr/lib/python3.6/concurrent/futures...  0.000076   
                            calculate_semblance::../seismicpro/batchflow/ba...  0.000062   
                            _action_wrapper::../seismicpro/batchflow/batchf...  0.000057   
     load #0                <method 'acquire' of '_thread.lock' objects>::/...  0.056172   
   

Funciton with one numba loop and matrix operations. Function ```_calc_semb_hard_numba_mx```.

In [9]:
pipeline_v2 = (
    dset.p
    .load(fmt='segy', components='raw', tslice=slice(1500))
    .sort_traces(src='raw', dst='raw', sort_by='offset')
    .calculate_semblance('raw', 'semblance_numba_matrix', [1200, 6000], 
                         30, window=51, method='numba_matrix')
)
pipeline_v2.run(1, n_iters=1, shuffle=1980, profile=True);

In [10]:
pipeline_v2.elapsed_time

8.129067182540894

In [11]:
pipeline_v2.show_profile_info(per_iter=True, detailed=False)

total_time  pipeline_time      batch_id
iter action                                                         
1    calculate_semblance #2    7.912481       7.898750  1.397954e+14
     load #0                   0.162511       0.140115  1.397954e+14
     sort_traces #1            0.025054       0.016523  1.397954e+14

In [12]:
pipeline_v2.show_profile_info(per_iter=True, detailed=True)

ncalls  \
iter action                 id                                                           
1    calculate_semblance #2 <method 'acquire' of '_thread.lock' objects>::/...       4   
                            <built-in method _thread.start_new_thread>::/us...       1   
                            wait::../seismicpro/batchflow/batchflow/decorat...       1   
                            <method 'acquire' of '_thread.lock' objects>::/...       1   
                            wrapped_method::../seismicpro/batchflow/batchfl...       1   
                            wrap_with_threads::../seismicpro/batchflow/batc...       1   
                            remove::../seismicpro/batchflow/batchflow/decor...       1   
                            wait::/usr/lib/python3.6/threading.py::533::wait         2   
                            calculate_semblance::../seismicpro/batchflow/ba...       1   
                            _call_post_fn::../seismicpro/batchflow/batchflo...       1   
     load #0                <method 'acquire' of '_thread.lock' objects>::/...       4   
                            <built-in method numpy.core._multiarray_umath.i...      26   
                            _load_segy::../seismicpro/src/seismic_batch.py:...       1   
                            <method 'get_loc' of 'pandas._libs.index.BaseMu...      93   
                            <built-in method builtins.getattr>::/usr/local/...    2187   
                            _check::~::0::<built-in method builtins.isinsta...    2187   
                            <method 'factorize' of 'pandas._libs.hashtable....      12   
                            <built-in method builtins.isinstance>::/usr/loc...     530   
                            _engine::/usr/local/lib/python3.6/dist-packages...       9   
                            find::/usr/local/lib/python3.6/dist-packages/pa...     328   
     sort_traces #1         <method 'acquire' of '_thread.lock' objects>::/...       4   
                            wait::../seismicpro/batchflow/batchflow/decorat...       1   
                            <method 'acquire' of '_thread.lock' objects>::/...       1   
                            <built-in method _thread.start_new_thread>::/us...       1   
                            wrap_with_threads::../seismicpro/batchflow/batc...       1   
                            wait::/usr/lib/python3.6/threading.py::533::wait         2   
                            wrapped_method::../seismicpro/src/seismic_batch...       1   
                            sort_traces::../seismicpro/src/seismic_batch.py...       1   
                            _call_post_fn::../seismicpro/batchflow/batchflo...       1   
                            remove::../seismicpro/batchflow/batchflow/decor...       1   

                                                                                 tottime  \
iter action                 id                                                             
1    calculate_semblance #2 <method 'acquire' of '_thread.lock' objects>::/...  7.896239   
                            <built-in method _thread.start_new_thread>::/us...  0.000378   
                            wait::../seismicpro/batchflow/batchflow/decorat...  0.000155   
                            <method 'acquire' of '_thread.lock' objects>::/...  0.000113   
                            wrapped_method::../seismicpro/batchflow/batchfl...  0.000104   
                            wrap_with_threads::../seismicpro/batchflow/batc...  0.000098   
                            remove::../seismicpro/batchflow/batchflow/decor...  0.000089   
                            wait::/usr/lib/python3.6/threading.py::533::wait    0.000086   
                            calculate_semblance::../seismicpro/batchflow/ba...  0.000084   
                            _call_post_fn::../seismicpro/batchflow/batchflo...  0.000072   
     load #0                <method 'acquire' of '_thread.lock' objects>::/...  0.080759   
   

One loop with pure numpy matrix operations. Function ```_calc_semb_hard_matrix```.

In [13]:
pipeline_v3 = (
    dset.p
    .load(fmt='segy', components='raw', tslice=slice(1500))
    .sort_traces(src='raw', dst='raw', sort_by='offset')
    .calculate_semblance('raw', 'semblance_matrix', [1200, 6000], 
                         30, window=51, method='matrix')
)
pipeline_v3.run(1, n_iters=1, shuffle=1980, profile=True);

In [14]:
pipeline_v3.elapsed_time

12.415731191635132

In [17]:
pipeline_v3.show_profile_info(per_iter=True, detailed=False)

total_time  pipeline_time      batch_id
iter action                                                         
1    calculate_semblance #2   12.192032      12.179128  1.397954e+14
     load #0                   0.176353       0.152676  1.397954e+14
     sort_traces #1            0.018305       0.011403  1.397954e+14

In [16]:
pipeline_v3.show_profile_info(per_iter=True, detailed=True)

ncalls  \
iter action                 id                                                           
1    calculate_semblance #2 <method 'acquire' of '_thread.lock' objects>::/...       4   
                            <method 'acquire' of '_thread.lock' objects>::/...       1   
                            wait::../seismicpro/batchflow/batchflow/decorat...       1   
                            <built-in method _thread.start_new_thread>::/us...       1   
                            wrap_with_threads::../seismicpro/batchflow/batc...       1   
                            wrapped_method::../seismicpro/batchflow/batchfl...       1   
                            _action_wrapper::../seismicpro/batchflow/batchf...       1   
                            _call_post_fn::../seismicpro/batchflow/batchflo...       1   
                            calculate_semblance::../seismicpro/batchflow/ba...       1   
                            remove::../seismicpro/batchflow/batchflow/decor...       1   
     load #0                <method 'acquire' of '_thread.lock' objects>::/...       4   
                            <built-in method numpy.core._multiarray_umath.i...      26   
                            _load_segy::../seismicpro/src/seismic_batch.py:...       1   
                            <built-in method builtins.getattr>::/usr/local/...    2187   
                            _check::~::0::<built-in method builtins.isinsta...    2187   
                            <method 'get_loc' of 'pandas._libs.index.BaseMu...      93   
                            <method 'factorize' of 'pandas._libs.hashtable....      12   
                            <built-in method builtins.isinstance>::/usr/loc...     530   
                            <built-in method numpy.array>::/usr/local/lib/p...     171   
                            find::/usr/local/lib/python3.6/dist-packages/pa...     328   
     sort_traces #1         <method 'acquire' of '_thread.lock' objects>::/...       4   
                            <built-in method _thread.start_new_thread>::/us...       1   
                            <method 'acquire' of '_thread.lock' objects>::/...       1   
                            wait::../seismicpro/batchflow/batchflow/decorat...       1   
                            wrap_with_threads::../seismicpro/batchflow/batc...       1   
                            wrapped_method::../seismicpro/src/seismic_batch...       1   
                            wait::/usr/lib/python3.6/threading.py::533::wait         2   
                            result::../seismicpro/batchflow/batchflow/decor...       1   
                            notify::/usr/lib/python3.6/queue.py::115::put            2   
                            pipeline::../seismicpro/batchflow/batchflow/bat...       2   

                                                                                  tottime  \
iter action                 id                                                              
1    calculate_semblance #2 <method 'acquire' of '_thread.lock' objects>::/...  12.177078   
                            <method 'acquire' of '_thread.lock' objects>::/...   0.000155   
                            wait::../seismicpro/batchflow/batchflow/decorat...   0.000111   
                            <built-in method _thread.start_new_thread>::/us...   0.000107   
                            wrap_with_threads::../seismicpro/batchflow/batc...   0.000093   
                            wrapped_method::../seismicpro/batchflow/batchfl...   0.000091   
                            _action_wrapper::../seismicpro/batchflow/batchf...   0.000082   
                            _call_post_fn::../seismicpro/batchflow/batchflo...   0.000081   
                            calculate_semblance::../seismicpro/batchflow/ba...   0.000077   
                            remove::../seismicpro/batchflow/batchflow/decor...   0.000069   
     load #0                <method 'acquire' of '_thread.lock' objects>::/...   0.

To conclude we can say that the fastest method is to use numba with 3 nested loops. 